#### Data 620 - Assignment 6<br>July 10, 2019<br>Team 2: <ul> <li>Anthony Munoz</li> <li>Katie Evers</li> <li>Juliann McEachern</li> <li>Mia Siracusa</li></ul>
<h1 align="center">"Document Classification"</h1>

It can be useful to be able to classify new "test" documents using already classified "training" documents.  A common example is using a corpus of labeled spam and ham (non-spam) e-mails to predict whether or not a new document is spam.  

Here is one example of such data:  [UCI Machine Learning Repository: Spambase Data Set](http://archive.ics.uci.edu/ml/datasets/Spambase).

For this project, you can either use the above dataset to predict the class of new documents (either withheld from the training dataset or from another source such as your own spam folder).
For more adventurous students, you are welcome (encouraged!) to come up a different set of documents (including scraped web pages!?) that have already been classified (e.g. tagged), then analyze these documents to predict how new documents should be classified.

This assignment is due end of day on Wednesday, July 10th.  You may work in a small team if you want.

*NOTE: This is a two week assignment.*

In [54]:
import pandas as pd
import numpy as np
import os


In [ ]:
path_ham = '/Users/Anthony/Desktop/School/MS/DATA-620/github/620/Assignments/data/ham/ham/'
email_list = os.listdir(path_ham)
path_spam = '/Users/Anthony/Desktop/School/MS/DATA-620/github/620/Assignments/data/spam/spam/'
email_list_spam = os.listdir(path_spam)
mylist = []  

### Data Uploading 

In [71]:
#The path for the ham and spam emails
path_ham = 'https://github.com/Anth350z/620/tree/master/Assignments/data/ham/ham'
email_list = os.listdir(path_ham)
path_spam = 'https://github.com/Anth350z/620/tree/master/Assignments/data/spam/spam/'
email_list_spam = os.listdir(path_spam)
mylist = []  

FileNotFoundError: [Errno 2] No such file or directory: 'https://github.com/Anth350z/620/tree/master/Assignments/data/ham/ham'

In [74]:
#require 'open-uri'
require 'net/http'

uri = "https://raw.githubusercontent.com/Anth350z/620/tree/master/Assignments/data/spam/spam/"
uri = URI(uri)
file = Net::HTTP.get(uri)

SyntaxError: invalid syntax (<ipython-input-74-cfa750edcc5e>, line 2)

In [56]:
#function to append the emails together on the list
def loading_data(email_type,mylist,path,email_list):
    #print(email_type,mylist,path,email_list)
    for i in email_list:
        with open(path + i , 'r', encoding="ISO-8859-1") as file:
           mylist.append([email_type , file.read()])
    
    

### Data Tagging

In [57]:
loading_data("ham",mylist,path_ham,email_list)
loading_data("spam",mylist,path_spam,email_list_spam)

In [58]:
#list length information
len(mylist)

2798

In [59]:
#convert the list to a dataframe with pandas
df_all = pd.DataFrame(mylist)
df_all.columns = ["label","email"]

In [60]:
df_all.head()

,label,email
0,ham,From rpm-list-admin@freshrpms.net Mon Jul 22 ...
1,ham,From webmake-talk-admin@lists.sourceforge.net ...
2,ham,From fork-admin@xent.com Wed Aug 14 11:01:15 ...
3,ham,From ilug-admin@linux.ie Mon Jul 22 19:50:20 ...
4,ham,From razor-users-admin@lists.sourceforge.net ...


### Data cleaning (Stemmer/Stopwords)

In [61]:
from nltk import stem
from nltk.corpus import stopwords

In [62]:
stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

In [63]:
#this function lower the data, stem and use stopwords for cleaning and organize the data.
def data_cleaning(data):
    data = data.lower()
    data = [message_word for message_word  in data.split() if message_word  not in stopwords]
    data = " ".join([stemmer.stem(message_word ) for message_word  in data])
    return data

In [64]:
#we call the function to apply some cleaning 
df_all['email'] = df_all['email'].apply(data_cleaning)


In [65]:
df_all.head()

,label,email
0,ham,rpm-list-admin@freshrpms.net mon jul 22 17:57:...
1,ham,webmake-talk-admin@lists.sourceforge.net tue a...
2,ham,fork-admin@xent.com wed aug 14 11:01:15 2002 r...
3,ham,ilug-admin@linux.i mon jul 22 19:50:20 2002 re...
4,ham,razor-users-admin@lists.sourceforge.net fri au...


### Spliting the Data 80/20

In [66]:
#we prepare and split the data for atrining 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

email = df_all['email'].values
label = df_all['label'].values
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                       stop_words='english')
email = vectorizer.fit_transform(email)

#training the data splitting with 80/20
X_train, X_test, y_train, y_test = train_test_split(email,label,test_size=0.2, shuffle=True,random_state=0,  stratify=label)

### Naive Bayes

In [67]:
#after the data is train we use the naive bayes for prediction model
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

print("Training Accuracy: " + str(clf.score(X_train, y_train)))

print(" Testing Accuracy: " + str(clf.score(X_test, y_test)))


Training Accuracy: 0.9857015192135835
 Testing Accuracy: 0.9803571428571428


### Random Forest Classifier

In [68]:
from sklearn.ensemble import RandomForestClassifier
RDC = RandomForestClassifier()
RDC.fit(X_train,y_train)

print("Training Accuracy: " + str(RDC.score(X_train, y_train)))

print(" Testing Accuracy: " + str(RDC.score(X_test, y_test)))

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training Accuracy: 0.9986595174262735
 Testing Accuracy: 0.9660714285714286


### Stochastic Gradient Descent (SGD)

In [69]:
from sklearn.linear_model import SGDClassifier

SGD = SGDClassifier()
SGD.fit(X_train,y_train)
print("Training Accuracy: " + str(SGD.score(X_train, y_train)))

print(" Testing Accuracy: " + str(SGD.score(X_test, y_test)))

Training Accuracy: 1.0
 Testing Accuracy: 0.9946428571428572


### References
1. https://www.datacamp.com/community/blog/text-mining-in-r-and-python-tips
2. https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk
3. https://towardsdatascience.com/spam-or-ham-introduction-to-natural-language-processing-part-2-a0093185aebd
4. https://gtraskas.github.io/post/spamit/